In [57]:
class Subposition:
    def __init__(self, stock_symbol, position_size):
        self.stock_symbol = stock_symbol
        self.position_size = position_size

    def __repr__(self):
        return f"Subposition({self.stock_symbol}, {self.position_size})"

class Position:
    def __init__(self, stock_symbol, cash_balance):
        self.stock_symbol = stock_symbol
        self.cash_balance = cash_balance
        self.subpositions = []

    def add_subposition(self, position_size):
        required_initial_margin = position_size / 2
        if required_initial_margin <= self.cash_balance:
            new_subposition = Subposition(self.stock_symbol, position_size)
            self.subpositions.append(new_subposition)
            self.cash_balance -= required_initial_margin
            print(f"Added: {new_subposition}")
        else:
            print("Insufficient initial margin to add subposition.")

    def total_position_size(self):
        return sum(subposition.position_size for subposition in self.subpositions)

    def __repr__(self):
        return f"Position(stock_symbol={self.stock_symbol}, cash_balance={self.cash_balance}, subpositions={self.subpositions})"


In [58]:
class Portfolio:
    def __init__(self, initial_cash_balance):
        self.total_cash_balance = initial_cash_balance
        self.positions = {}
        self.total_initial_margin_used = 0

    def add_position(self, stock_symbol, position_size):
        if stock_symbol not in self.positions:
            self.positions[stock_symbol] = Position(stock_symbol, self.total_cash_balance)

        position = self.positions[stock_symbol]
        required_initial_margin = position_size / 2

        # Calculate how much margin is available
        available_margin = self.total_cash_balance * 2 - self.total_initial_margin_used
        print(available_margin, required_initial_margin)
        
        if required_initial_margin <= available_margin:
            # While loop for splitting positions into subpositions
            while position_size > 2 * self.total_cash_balance:
                sub_position_size = 2 * self.total_cash_balance
                position.add_subposition(sub_position_size)
                self.total_initial_margin_used += sub_position_size / 2
                position_size -= sub_position_size

            if position_size > 0:
                position.add_subposition(position_size)
                self.total_initial_margin_used += position_size / 2
        else:
            print(f"Insufficient total initial margin to add position in {stock_symbol}.")

    def total_portfolio_value(self):
        return sum(position.total_position_size() for position in self.positions.values())

    def __repr__(self):
        return f"Portfolio(total_cash_balance={self.total_cash_balance}, positions={self.positions})"

In [59]:

# Example usage
initial_cash_balance = 100000

# Create a Portfolio instance
portfolio = Portfolio(initial_cash_balance)

# Add positions for multiple stocks
portfolio.add_position('AAPL', 300000)  # Should split into sub-positions
portfolio.add_position('GOOGL', 100000)  # Fits within the 2x limit


200000 150000.0
Added: Subposition(AAPL, 200000)
Insufficient initial margin to add subposition.
50000.0 50000.0
Added: Subposition(GOOGL, 100000)


In [60]:

# Display the portfolio and remaining cash balance
print(portfolio)
print(f"Total portfolio value: ${portfolio.total_portfolio_value()}")
print(f"Remaining total cash balance: ${portfolio.total_cash_balance}")
print(f"Total initial margin used: ${portfolio.total_initial_margin_used}")

Portfolio(total_cash_balance=100000, positions={'AAPL': Position(stock_symbol=AAPL, cash_balance=0.0, subpositions=[Subposition(AAPL, 200000)]), 'GOOGL': Position(stock_symbol=GOOGL, cash_balance=50000.0, subpositions=[Subposition(GOOGL, 100000)])})
Total portfolio value: $300000
Remaining total cash balance: $100000
Total initial margin used: $200000.0
